# MSSQL Integration Demo

### Imports & Setup

In [15]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, execute_script_file, display_result_sets

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup

In [2]:
connection_name = "DATABRIDGE"
peril = "USEQ"
edm_name = f"{peril}_Automated"
portfolio_name = f"{peril}_Automated"
date_value = "202511"

### Portfolio Mapping

In [3]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
exposure_id = edm['exposureId']
edm_full_name = edm['databaseName']
print(f'EDM Full Name: {edm_full_name}')

portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
if (len(portfolios) != 1):
    raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
portfolio_id = portfolios[0]['portfolioId']
print(f'Portfolio ID: {portfolio_id}')
print()

sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{peril}_RMS_BackEnd.sql"
params = {
    "EDM_FULL_NAME": edm_full_name,
    "PORTFOLIO_ID": portfolio_id,
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

result = execute_script_file(
    file_path = f'portfolio_mapping/{sql_script_name}',
    params = params,
    connection = connection_name
)

print(result)

EDM Full Name: USEQ_Automated_rGLG
Portfolio ID: 1

Reading SQL script ...
Parameterizing ...
Executing SQL ...
Script Execution Result: ✓ SUCCESS
  Statements executed: 1
  Rows affected:       0
  Result sets:         0


### Control Totals

In [16]:
workspace_edm = "WORKSPACE_EDM"
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
edm_full_name = edm['databaseName']
print(f'Workspace EDM Full Name: {edm_full_name}')
print()

sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": edm_full_name,
    "CYCLE_TYPE": "Quarterly",
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

# Display results in a clean format instead of print(result)
display_result_sets(result)

Workspace EDM Full Name: USEQ_Automated_rGLG

Reading SQL script ...
Parameterizing ...
Executing SQL query: 3d_RMS_EDM_CONTROL_Totals.sql
✓ Retrieved 10 result set(s)

QUERY RESULTS: 10 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 6



,PORTNAME,PolicyCount,PolicyLimit,PolicyPremium,AttachmentPoint,PolicyDeductible
0,USEQ_Automated,516777,61568158484.00,565524244.00,0.00,0.00
1,USEQ_CHFS,344,362956599.00,1359514.00,0.00,0.00
2,USEQ_Clay_21st,148768,17232894141.00,158768016.00,0.00,0.00
3,USEQ_Clay_Homes,136315,19141446773.00,170120715.00,0.00,0.00
4,USEQ_Condo,144,21756768.00,149376.00,0.00,0.00
5,USEQ_Manufactured,196767,23973540757.00,227029441.00,0.00,0.00
6,USEQ_Renters,34439,835563446.00,8097182.00,0.00,0.00



--------------------------------------------------------------------------------
Result Set 2 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 3



,PORTNAME,LocationCountDistinct,LocationCountCampus
0,USEQ_Automated,516777,516777
1,USEQ_CHFS,344,344
2,USEQ_Clay_21st,148768,148768
3,USEQ_Clay_Homes,136315,136315
4,USEQ_Condo,144,144
5,USEQ_Manufactured,196767,196767
6,USEQ_Renters,34439,34439



--------------------------------------------------------------------------------
Result Set 3 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['PORTNAME', 'TotalReplacementValue', 'LocationLimit']

--------------------------------------------------------------------------------
Result Set 4 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 2



,PORTNAME,LocationDeductible
0,USEQ_Automated,0.00
1,USEQ_CHFS,0.00
2,USEQ_Clay_21st,0.00
3,USEQ_Clay_Homes,0.00
4,USEQ_Condo,0.00
5,USEQ_Manufactured,0.00
6,USEQ_Renters,0.00



--------------------------------------------------------------------------------
Result Set 5 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['PORTNAME', 'PolicyCount', 'PolicyPremium', 'PolicyLimit_NonCommercialFlood', 'AttachmentPoint', 'PolicyDeductible']

--------------------------------------------------------------------------------
Result Set 6 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['USFL_Commercial_PolicyLimit']

--------------------------------------------------------------------------------
Result Set 7 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['PORTNAME', 'Policy_Sublimit']

--------------------------------------------------------------------------------
Result Set 8 of 10
--------------------------------------------------------------------------------
Empty DataFrame

### Execute Simple SQL Scripts

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "list_edm_tables.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df)

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "test_multiple_select.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df.head())